In [1]:
import torch
import torch.nn as nn
from torch.nn import Embedding

import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import dataset
import test_lstm

2023-03-11 12:34:38.320312: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
lie_trial_path = './processed_lie/' #60 entries
truth_trial_path = './processed_truth/' #61 entries
MU3D_path = './processed/' # 300 entries

In [3]:
# no split by person
numOfFrames = 10
X, Y = dataset.preprocessing(truth_trial_path, lie_trial_path, numOfFrames=numOfFrames)

TEST_RATIO = 0.2

xTrain, xTest = train_test_split(X, test_size=TEST_RATIO, shuffle=False)
yTrain, yTest = train_test_split(Y, test_size=TEST_RATIO, shuffle=False)

In [46]:
class positionalEncoder(nn.Module):

  def __init__(self, frame_length, encoding_length):
    super().__init__()

    embedding = nn.Embedding(frame_length, encoding_length)

    self.pe = embedding(torch.tensor([i for i in range(frame_length)]).unsqueeze(1)).squeeze()

  def forward(self, x):

    embedded = []

    if x.dim() == 3:
      for i in range(x.shape[0]):
        embedded.append(torch.cat((x[i], self.pe), 1).detach().numpy())
    else:
      print('x.shape = ', x.shape)
      print('self.pe.shape = ', self.pe.shape)
      embedded = torch.cat((x, self.pe), 1).detach().numpy()
      
    return torch.tensor(embedded)

class classifierNetwork(nn.Module):

  def __init__(self, inFeatCount, num_T_layers, num_frames, device, pos_encode_size = 5, n_heads = 4, n_hidden = 2048, dropout = 0.3, outFeatCount = 2):
    super().__init__()

    self.posEncoder = positionalEncoder(num_frames, pos_encode_size)

    heads = n_heads
    num_features = inFeatCount + pos_encode_size

    if (num_features % heads) != 0:
      heads += heads - (num_features % heads)

    print(f'features = {num_features}, heads = {heads}')

    n_hidden = max(n_hidden, 2*num_features)

    encoder_layer = nn.TransformerEncoderLayer(inFeatCount + pos_encode_size, heads, n_hidden, dropout)
    self.encoder = nn.TransformerEncoder(encoder_layer, num_T_layers)
    
    many_to_one_feat = num_frames * num_features
    mid = (many_to_one_feat - outFeatCount) // 2 + outFeatCount

    self.fc1 = nn.Linear(many_to_one_feat, mid)
    self.fc2 = nn.Linear(mid, 2)

    self.device = device

    self.init_weights()

  def init_weights(self):
      initrange = 0.1
      self.fc1.bias.data.zero_()
      self.fc1.weight.data.uniform_(-initrange, initrange)

      self.fc2.bias.data.zero_()
      self.fc2.weight.data.uniform_(-initrange, initrange)

  def forward(self, x):
    #x.shape = [num_frames, feat_count]
    encoded = self.posEncoder(x)
    #encoded.shape = [num_frames, feat_count + pos_encoding_count]
    data = self.encoder(encoded)
    #data.shape = [num_frames, feat_count + pos_encoding_count]
    data = torch.reshape(data, (1,-1))
    #data.shape = [1, num_frames * (feat_count + pos_encoding_count)] 

    data = self.fc1(data)
    data = self.fc2(data)
    data = nn.functional.softmax(data, dim = 1).to(self.device)      

    return torch.tensor(data.tolist()[0])

In [48]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

featCount = 10
num_frames = 10
encoder_layers = 2

net = classifierNetwork(featCount, encoder_layers, num_frames, device)

yTrain_temp, yTest_temp = [], []

for i in range(yTrain.shape[0]):
    yTrain_temp.append([1,0]) if yTrain[i] == 0 else yTrain_temp.append([0,1])

for i in range(yTest.shape[0]):
    yTest_temp.append([1,0]) if yTest[i] == 0 else yTest_temp.append([0,1])

y_Train = torch.tensor(yTrain_temp).to(device)
y_Test = torch.tensor(yTest_temp).to(device)

x_Train = torch.tensor(xTrain, dtype=torch.float32).to(device)
x_Test = torch.tensor(xTest, dtype=torch.float32).to(device)

features = 15, heads = 5


In [43]:
pred = net(x_Train[0])
print(pred)

x.shape =  torch.Size([10, 10])
self.pe.shape =  torch.Size([10, 5])
tensor([0.4134, 0.5866])


In [51]:
# training
import torch.optim as optim

def train(model, xTrain, yTrain, xTest, yTest, epochs = 100, lr = 0.001, batch_size = 10):
    """ Train a model on a dataset """

    loss_items = []
    accuracy_items = []
    test_accuracy_items = []
    
    # create a data loader to handle batching
    xTrain_loader =  torch.utils.data.DataLoader(xTrain, batch_size=batch_size, shuffle=False)
    xTest_loader = torch.utils.data.DataLoader(xTest, batch_size=batch_size, shuffle=False)

    # create a loss function and optimizer
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # train the model
    for epoch in range(epochs):

        # train

        idx = 0
        model.train()
        for batch in torch.tensor(xTrain):

            # get data
            x_train = batch.to(device).float()
            y_train = torch.tensor(yTrain[idx]).to(device).float()

            # forward pass
            y_pred = model(x_train)

            # compute loss
            loss = loss_fn(y_pred, y_train)

            # backward pass
            loss.backward()

            # update weights
            optimizer.step()
            optimizer.zero_grad()

        # evaluate
        model.eval()

        test_acc = 0
        acc = 0
        tot_loss = 0

        with torch.no_grad():
            
            idx_test = 0
            for batch in torch.tensor(xTest):
                xTest = batch.to(device).float()
                y_pred = model(xTest)
                y_test = torch.tensor(yTest[idx_test]).to(device)

                # compute test accuracy
                test_acc += (y_pred.argmax() == y_test.argmax()).type(torch.float).mean().item()
                idx_test += 1
                

            test_acc /= len(xTest)  
            test_accuracy_items.append(test_acc)

            idx_train = 0
            for batch in torch.tensor(xTrain):
                xTrain = batch.to(device).float()
                y_pred = model(xTrain)
                y_train = torch.tensor(yTrain[idx_train]).to(device)

                # compute accuracy
                acc += (y_pred.argmax(0) == y_train).type(torch.float).mean().item()
                idx_train += 1

                # computer loss
                tot_loss += loss_fn(y_pred, y_train).item()

            acc /= len(xTrain)
            accuracy_items.append(acc)

        # store loss and accuracy
        loss_items.append(tot_loss)

        # print progress
        if (epoch+1) % 1 == 0:
            print(f'Epoch {epoch+1}/{epochs}, Loss: {tot_loss:.4f}, Accuracy: {acc:.4f}, Test Accuracy: {test_acc:.4f}')

        idx += 1
            

train(net, xTrain, y_Train, xTest, y_Test)

x.shape =  torch.Size([10, 10])
self.pe.shape =  torch.Size([10, 5])
tensor([0.3694, 0.6306])
tensor([0., 1.])


/var/folders/66/pn8653k51_75x5ksbpsy8hm80000gn/T/ipykernel_26121/1599639121.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(yTrain[idx]).to(device).float()


TypeError: cross_entropy_loss(): argument 'input' (position 1) must be Tensor, not list